In [16]:
%%configure -f
{
    "driverCores": 1,
    "conf":{
    "spark.hadoop.fs.dtap.impl":"com.bluedata.hadoop.bdfs.Bdfs",
    "spark.hadoop.fs.AbstractFileSystem.dtap.impl":"com.bluedata.hadoop.bdfs.BdAbstractFS",
    "spark.hadoop.fs.dtap.impl.disable.cache": "false",
    "spark.driver.extraClassPath":"/opt/bdfs/bluedata-dtap.jar",
    "spark.executor.extraClassPath": "/opt/bdfs/bluedata-dtap.jar",
    "spark.kubernetes.driver.label.hpecp.hpe.com/dtap":"hadoop2",
    "spark.kubernetes.executor.label.hpecp.hpe.com/dtap":"hadoop2",
    "spark.kubernetes.executor.limit.cores":5,
    "spark.driver.memory":"20g",
    "spark.executor.memory": "40g",
    "spark.executor.cores":5,
    "spark.executor.instances":4,
    "spark.kubernetes.container.image":"smtds/spark-py-2.4.7-oracle:20230330",
    "spark.jars": "local:///opt/mapr/spark/spark-py-2.4.7/external_jars/ojdbc8-21.9.0.0.jar"
    }
}

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
Time taken to configure Spark Session: 97.80365180969238 seconds


In [ ]:
%%spark

In [17]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import (
    StructType,
    StringType,
    StructField,
)
import itertools
from pyspark.sql.window import Window
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import SparkSession
from itertools import chain
from functools import partial, reduce

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
spark.read.parquet('dtap://ext_mapr/ws-michael/temp/cnss202310.parquet').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

984575

In [12]:
from enum import Enum


class AppCategoryLv1(Enum):
    STREAMING = {
        "afreeca",
        "bilibili",
        "disney_plus",
        "facebook_live",
        "facebook_video",
        "netflix",
        "qqvideo",
        "viu",
        "youku",
        "youtube",
        "youtube_kids",
    }
    MUSIC_STREAMING = {
        "apple_music",
        "kkbox",
        "qqmusic",
        "spotify",
        "spotify_audio",
        "youtube_music",
    }
    COMMUNICATION = {
        "discord",
        "facebook_messenger",
        "line",
        "signal_private_messenger",
        "telegram",
        "wechat",
        "whatsapp",
        "whatsapp_api",
    }
    PAYMENT = {"alipay", "apple_pay", "google_pay", "samsung_pay"}
    VPN = {
        "expressvpn",
        "hideman_vpn",
        "hot_vpn",
        "openvpn",
        "opera_vpn",
        "snap_vpn",
        "turbo_vpn",
    }
    SOCIAL_NETWORK = {
        "facebook",
        "instagram",
        "linkedin",
        "sina_weibo",
        "twitter",
        "xiaohongshu",
    }
    ECOMMERCE = {
        "amazon",
        "ebay",
        "jingdong",
        "pinduoduo",
        "rakuten",
        "taobao",
        "taote",
        "tmall",
    }
    GAMING = {
        "battlenet",
        "call_of_duty",
        "candy_crush_saga",
        "clash_of_clans",
        "clash_royale",
        "minecraft",
        "pokemon_go",
        "pubg",
        "roblox",
        "steam"
    }
    SPEED_TEST = {"fast_com"}
    RETAIL = {"ikea"}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def _fill_na_count_column_as_zero(df: DataFrame) -> DataFrame:
    fill_values = {
        c: 0
        for c in [_.lower() for _ in df.columns]
        if "_count" in c or "_data_usage" in c or "_duration_days" in c
    }
    df = df.fillna(fill_values)

    return df


def _rename_df_columns(
    sdf: DataFrame,
    prefix: str = "",
    suffix: str = "",
    excluded_list: list = ["CUST_NUM", "SUBR_NUM"],
) -> DataFrame:
    if not (prefix or suffix):
        return sdf

    for c in sdf.columns:
        if c in excluded_list:
            continue
        sdf = sdf.withColumnRenamed(c, prefix + c + suffix)
    return sdf


def _app_categories_groups(spark: SparkSession):
    ### ---------------------- layer 1 ------------------------ ###
    # get list of apps
    apps = set.union(*[c.value for c in AppCategoryLv1])

    # get cat, app generators
    rows = [itertools.product([cat.name], cat.value) for cat in AppCategoryLv1]
    # chain all generators
    rows = chain.from_iterable(rows)

    # create spark dataframe for tagging
    layer1_df = spark.createDataFrame(
        rows,
        schema=StructType(
            [
                StructField("APP_CAT_LV1", StringType()),
                StructField("APP_NAME", StringType()),
            ]
        ),
    )

    app_cat = layer1_df
    return app_cat


def clean_cnss_activities(
    spark: SparkSession,
    run_date: datetime,
    sdf_cnss_raw: DataFrame,
    sdf_post_paid_trx: DataFrame,
) -> DataFrame:
    # read VW_BM_LIS_ROAM_COUNT
    this_month = run_date.replace(day=1)

    sdf_cnss_raw = (
        sdf_cnss_raw.withColumnRenamed("MSISDN", "SUBR_NUM")
        .filter(col("USG_MB") > 0.0)
        .filter(F.trim(col("SUBR_NUM")) != "")
        .filter(F.length(col("SUBR_NUM")) == 11)
        .filter(col("SUBR_NUM").startswith("852"))
        .withColumn("SUBR_NUM", F.substring(col("SUBR_NUM"), 4, 8))
        .withColumn(
            "APP_NAME",
            F.when(col("APP_NAME").isNull(), col("SERVICE_NAME")).otherwise(
                col("APP_NAME")
            ),
        )
        .drop("SERVICE_NAME")
    )

    # combine apps into one
    # added on 20230505
    sdf_cnss_raw = sdf_cnss_raw.withColumn(
        "APP_NAME",
        F.when(col("APP_NAME") == "youtube_hd", "youtube")
        .when(col("APP_NAME") == "netflix_video", "netflix")
        .otherwise(col("APP_NAME")),
    )

    # group to prevent multiple records on the same day
    sdf_cnss_raw = sdf_cnss_raw.groupby(["SUBR_NUM", "APP_NAME", "ACCS_DATE"]).agg(
        F.sum(col("USG_MB")).alias("USG_MB")
    )

    # list of customer in this month
    sdf_post_paid_trx_cm = sdf_post_paid_trx.select("CUST_NUM", "SUBR_NUM").filter(
        col("MONTH_ID") == F.lit(this_month.strftime("%Y-%m-%d"))
    )

    # spark df of the app category class
    sdf_app_cat = _app_categories_groups(spark)

    # join all functions
    sdf_cleaned = sdf_cnss_raw.join(
        F.broadcast(sdf_post_paid_trx_cm), on=["SUBR_NUM"], how="inner"
    ).join(F.broadcast(sdf_app_cat), on=["APP_NAME"], how="inner")

    return sdf_cleaned


def _get_features_pivot(
    sdf: DataFrame,
    run_date: datetime,
    pivot_col: str,
    pivot_values: list,
    feature_col_prefix: str,
    exclude_list: list = [],
    cnss_last_n_months: int = 3,
) -> DataFrame:
    ## get feature suffix
    if cnss_last_n_months == 1:
        feature_col_suffix = f"_LM"
    else:
        feature_col_suffix = f"_{cnss_last_n_months}M"

    if cnss_last_n_months < 3:
        start = (
            run_date.replace(day=1) - relativedelta(months=cnss_last_n_months - 1)
        ).strftime("%Y-%m-%d")
        sdf = sdf.filter(col("ACCS_DATE") >= F.lit(start))

    # filter excluded apps
    if exclude_list:
        sdf = sdf.filter(~col(pivot_col).isin(exclude_list))
        pivot_values = list(set(pivot_values) - set(exclude_list))

    def _by_session(sdf: DataFrame):
        window = Window.partitionBy("CUST_NUM", "SUBR_NUM", pivot_col).orderBy(
            "CUST_NUM", "SUBR_NUM", pivot_col, "ACCS_DATE"
        )

        # sum of data usage per session/day
        # number of consecutive days
        _sdf_sessions = (
            sdf.withColumn(
                "days_since_last_hit_date",
                F.datediff(col("ACCS_DATE"), F.lag(col("ACCS_DATE")).over(window)),
            )
            .withColumn(
                "is_session_start",
                F.when(col("days_since_last_hit_date").isNull(), 1)
                .when(col("days_since_last_hit_date") > 1, 1)
                .otherwise(0),
            )
            .withColumn("session_id", F.sum(col("is_session_start")).over(window))
            .groupby(
                "CUST_NUM",
                "SUBR_NUM",
                pivot_col,
                "session_id",
            )
            .agg(
                (
                    F.datediff(F.max(col("ACCS_DATE")), F.min(col("ACCS_DATE"))) + 1
                ).alias("session_duration_days"),
                F.sum(col("USG_MB")).alias("session_data_usage"),
            )
        )

        # category pivoting
        _sdf_sessions = (
            _sdf_sessions.groupby("CUST_NUM", "SUBR_NUM")
            .pivot(pivot_col, values=pivot_values)
            .agg(F.max(col("session_duration_days")))
        )

        return _rename_df_columns(
            _sdf_sessions,
            prefix=feature_col_prefix,
            suffix="_MAX_DURATION_DAYS" + feature_col_suffix,
        )

    def _by_wdwe(sdf: DataFrame):
        # weekend indicator
        _sdf_wdwe = sdf.withColumn(
            "wdwe",
            F.when(F.dayofweek("ACCS_DATE").isin([1, 7]), "weekend").otherwise(
                "weekday"
            ),
        )
        # temp pivot column
        _sdf_wdwe = _sdf_wdwe.withColumn(
            "temp_pivot_col", F.concat_ws("_", col(pivot_col), col("wdwe"))
        )
        # get pivot values for wdwe features
        pivot_values_wdwe = [
            "_".join(pair)
            for pair in itertools.product(pivot_values, ["weekday", "weekend"])
        ]

        # pivoting
        _sdf_wdwe = (
            _sdf_wdwe.groupby("CUST_NUM", "SUBR_NUM")
            .pivot("temp_pivot_col", values=pivot_values_wdwe)
            .agg(
                F.count(col("SUBR_NUM")).alias("DAYS_COUNT" + feature_col_suffix),
                F.round(F.sum(col("USG_MB")), 3).alias(
                    "DATA_USAGE" + feature_col_suffix
                ),
            )
        )

        return _rename_df_columns(_sdf_wdwe, prefix=feature_col_prefix)

    def _general(sdf: DataFrame):
        # weekend indicator
        _sdf = (
            sdf.groupby("CUST_NUM", "SUBR_NUM")
            .pivot(pivot_col, values=pivot_values)
            .agg(
                F.count(col("SUBR_NUM")).alias("DAYS_COUNT" + feature_col_suffix),
                F.round(F.sum(col("USG_MB")), 3).alias(
                    "DATA_USAGE" + feature_col_suffix
                ),
            )
        )

        return _rename_df_columns(_sdf, prefix=feature_col_prefix)

    # list of features
    feature_dfs = [_by_session(sdf), _by_wdwe(sdf), _general(sdf)]

    def _join_func(a, b):
        return a.join(b, on=["CUST_NUM", "SUBR_NUM"], how="outer")

    # join feature dfs into one
    all_features = reduce(_join_func, feature_dfs)
    all_features = _fill_na_count_column_as_zero(all_features)

    # col name to upper
    for c in all_features.columns:
        all_features = all_features.withColumnRenamed(c, c.upper())

    return all_features
    # return _sdf_feature_session, _sdf_feature_wdwe


def _get_features_distinct_count(
    sdf: DataFrame,
    run_date: datetime,
    pivot_col: str,
    pivot_values: list,
    count_col: str,
    feature_col_prefix: str,
    pivot_exclude_list: list = [],
    count_exclude_list: list = [],
    cnss_last_n_months: int = 3,
):
    ## get feature suffix
    if cnss_last_n_months == 1:
        feature_col_suffix = f"_LM"
    else:
        feature_col_suffix = f"_{cnss_last_n_months}M"

    if cnss_last_n_months < 3:
        start = (
            run_date.replace(day=1) - relativedelta(months=cnss_last_n_months - 1)
        ).strftime("%Y-%m-%d")
        sdf = sdf.filter(col("ACCS_DATE") >= F.lit(start))

    # filter pivot values
    if pivot_exclude_list:
        sdf = sdf.filter(~col(pivot_col).isin(pivot_exclude_list))
        pivot_values = list(set(pivot_values) - set(pivot_exclude_list))

    # filter count values
    if count_exclude_list:
        sdf = sdf.filter(~col(count_col).isin(count_exclude_list))

    # count distinct
    _sdf_dc = (
        sdf.groupby("CUST_NUM", "SUBR_NUM")
        .pivot(pivot_col, values=pivot_values)
        .agg(
            F.countDistinct(col(count_col)),
        )
    )

    # col name to upper
    for c in _sdf_dc.columns:
        _sdf_dc = _sdf_dc.withColumnRenamed(c, c.upper())

    return _fill_na_count_column_as_zero(
        _rename_df_columns(
            _sdf_dc,
            prefix=feature_col_prefix,
            suffix="_DISTINCT_COUNT" + feature_col_suffix,
        )
    )


def calculate_features(
    spark: SparkSession,
    # injected from config.yaml:
    target_month: str = "1990-01",
) -> DataFrame:
    """

    Args:
        spark:
        target_month:
        raw_input_dir:
        cleansed_dir:
        **kwargs:

    Returns:

    """

    run_date = datetime.strptime(target_month, "%Y-%m")

    def _paths_for_partition_in_days(
        path_format: str, run_date: datetime, prev_n_months: int = 1
    ) -> list:
        # last day
        last_day = run_date + relativedelta(months=1) + relativedelta(days=-1)

        # get number of days in total for prev_n_months
        total_days = (
            last_day - (run_date + relativedelta(months=-(prev_n_months - 1)))
        ).days + 1

        paths = [
            (last_day + relativedelta(days=-i)).strftime(path_format)
            for i in range(total_days)
        ]
        return paths

    # raw data available date
    raw_data_cutoff_date = datetime.strptime("2022-12", "%Y-%m")
    # diff in months, will read data depends on the difference in months
    month_diff = (
        (run_date.year - raw_data_cutoff_date.year) * 12
        + (run_date.month - raw_data_cutoff_date.month)
        + 1
    )

    # list of unfiltered feature months list
    feature_month_cutoff_list = [1, 3, 6, 9, 12]
    # filter with month diff
    feature_month_cutoff_list = [
        i for i in feature_month_cutoff_list if i <= month_diff
    ]

    ## --------------------------- Read Data - CNSS -------------------------- ##
    paths = _paths_for_partition_in_days(
        "dtap://ext_mapr/c360_data/BP.VW_CNSS_DAILY_SUMM/ACCS_DATE=%Y-%m-%d",
        run_date,
        prev_n_months=feature_month_cutoff_list[-1],
    )

    sdf_cnss_raw = spark.read.option(
        "basePath", "dtap://ext_mapr/c360_data/BP.VW_CNSS_DAILY_SUMM"
    ).parquet(*paths)

    ## --------------------------- Read Data - N1 --------------------------- ##
    sdf_post_paid_trx_cm = spark.read.parquet(
        f'dtap://ext_mapr/c360-pipeline/cleansed/{target_month}/PRD_BIZ_SUMM_VW.VW_SHK_POSTPAID_MASTER_SUMM'
    )

    ## -------------------------- Clean CNSS Data -------------------------- ##
    sdf_cnss = clean_cnss_activities(
        spark,
        run_date,
        sdf_cnss_raw,
        sdf_post_paid_trx_cm,
    )

    ## --------------------------- Get Features --------------------------- ##
    # app features
    get_features_pivot_app = partial(
        _get_features_pivot,
        sdf=sdf_cnss,
        run_date=run_date,
        pivot_col="APP_NAME",
        pivot_values=list(
            reduce(lambda a, b: a.union(b), [a.value for a in AppCategoryLv1])
        ),
        feature_col_prefix="CNSS_APP_",
        exclude_list=list(AppCategoryLv1.VPN.value),
    )
    # category features
    get_features_pivot_category = partial(
        _get_features_pivot,
        sdf=sdf_cnss,
        run_date=run_date,
        pivot_col="APP_CAT_LV1",
        pivot_values=[a.name for a in AppCategoryLv1],
        feature_col_prefix="CNSS_CATEGORY_",
    )
    # app distinct count
    get_features_distinct_count_app = partial(
        _get_features_distinct_count,
        sdf=sdf_cnss,
        run_date=run_date,
        pivot_col="APP_CAT_LV1",
        pivot_values=[a.name for a in AppCategoryLv1],
        count_col="APP_NAME",
        feature_col_prefix="CNSS_CATEGORY_",
    )

    def _get_features_list_by_month(month):
        cnss_features = [
            get_features_pivot_app(cnss_last_n_months=month),
            get_features_pivot_category(cnss_last_n_months=month),
            get_features_distinct_count_app(cnss_last_n_months=month),
        ]
        return cnss_features

    # get list of all features
    all_cnss_features = reduce(
        lambda x, y: x + y,
        [_get_features_list_by_month(m) for m in feature_month_cutoff_list],
    )

    def _join_func(a, b):
        return a.join(b, on=["CUST_NUM", "SUBR_NUM"], how="outer")

    features = reduce(_join_func, all_cnss_features).withColumn(
        "MONTH_ID", F.to_date(F.lit(run_date.replace(day=1).strftime("%Y-%m-%d")))
    )
    features = _fill_na_count_column_as_zero(features)
    return features

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def cnss_app_usage(spark: SparkSession, target_month: str = "1990-01"):
   
    return calculate_features(spark, target_month)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
cnss_app_usage(spark, "2023-03").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

972031

In [ ]:
cnss_app_usage(spark, "2023-09")

In [9]:
cnss_app_usage(spark, "2023-09").write.mode('overwrite').parquet('dtap://ext_mapr/ws-michael/temp/cnss202309.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
cnss_app_usage(spark, "2023-10").write.mode('overwrite').parquet('dtap://ext_mapr/ws-michael/temp/cnss202310.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
run_date= datetime.strptime('2023-10', "%Y-%m")

feature_month_cutoff_list = [1, 3, 6, 9, 12]

def _paths_for_partition_in_days(
        path_format: str, run_date: datetime, prev_n_months: int = 1
    ) -> list:
        # last day
        last_day = run_date + relativedelta(months=1) + relativedelta(days=-1)

        # get number of days in total for prev_n_months
        total_days = (
            last_day - (run_date + relativedelta(months=-(prev_n_months - 1)))
        ).days + 1

        paths = [
            (last_day + relativedelta(days=-i)).strftime(path_format)
            for i in range(total_days)
        ]
        return paths
    
paths = _paths_for_partition_in_days(
        "dtap://ext_mapr/c360_data/BP.VW_CNSS_DAILY_SUMM/ACCS_DATE=%Y-%m-%d",
        run_date,
        prev_n_months=feature_month_cutoff_list[-1],
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
paths[5]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'dtap://ext_mapr/c360_data/BP.VW_CNSS_DAILY_SUMM/ACCS_DATE=2023-10-26'

In [17]:
spark.read.parquet("dtap://ext_mapr/weblog/t_domain_dictionary").filter(F.col('tag_gp')=='Travel').select('tag').distinct().show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+
|tag                        |
+---------------------------+
|Car Transportation         |
|Online Travel Agents       |
|Ground/Cruise              |
|Hotel/Resort & Home Sharing|
|Travel - Other             |
|Airlines                   |
|Travel - Information       |
+---------------------------+

In [13]:
spark.read.parquet("dtap://ext_mapr/weblog/t_domain_dictionary").select('tag').distinct().show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+
|tag                        |
+---------------------------+
|Personal Finance           |
|Car Transportation         |
|Home/Architecture          |
|Health Care                |
|Cyber Security             |
|Hardcore Gamer             |
|Corporate Presence - Other |
|Fragrances/Cosmetics       |
|Computer Hardware          |
|Discussion/Chat            |
|Sports/Outdoor             |
|Food/Supermarket/Grocery   |
|GOVERNMENT PRIMARY SCHOOLS |
|Entertainment - News       |
|Information                |
|Promotional Servers - Other|
|Entertainment - Other      |
|Forums/Message Boards      |
|Insurance                  |
|Payments                   |
+---------------------------+
only showing top 20 rows

In [13]:
cnss_app_usage(spark, "2023-04").write.format("parquet").save('dtap://ext_mapr/ws-michael/temp/cnss202304.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark.read.parquet('dtap://ext_mapr/ws-michael/temp/cnss202304.parquet').select('CNSS_APP_SPOTIFY_WEEKEND_DAYS_COUNT_12M').show()


In [28]:
col = spark.read.parquet('dtap://ext_mapr/ws-michael/temp/cnss202302.parquet').columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
col2

In [29]:
col2 = spark.read.parquet('dtap://ext_mapr/c360-pipeline/v3_docstring_test/derived/cnss_app_usage/cnss_app_usage/2023-05.parquet').columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
len(set(col2)-set(col))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

465

In [35]:
cnt=0
for i in set(col2)-set(col):
    if '_6M' in i:
        cnt += 1
print(cnt)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

465